#Using Balance Theory to detect driver genes in cancer

We have an idea to detect driver genes in cancer using balance theory. We know that every complex system is in minimum free energy state and tends to stay there. Pertubrations cause the system to unbalance and the free energy to increases. As the system tries to cool down to minimal energy, some changes happen. We are going to use this concept to detect driver genes that cause cancer, as they probably change free energy of the system.

To this end, we use Prostate cancer data derived from NCBI Gene Omnibus GDS2545 dataset. We first load data of prostate tissue in four states: Normal, Adjacent, Tumor, Metastasis.

In [ ]:
import numpy as np

# Use dtype=None to have string values as are, otherwise numpy tries to convert them into fload.
normal_data = np.genfromtxt('normal.csv',delimiter=',',names=True,dtype=None)
adjacent_data = np.genfromtxt('adjacent.csv',delimiter=',',names=True,dtype=None)
tumor_data = np.genfromtxt('tumor.csv',delimiter=',',names=True,dtype=None)
metastasis_data = np.genfromtxt('metastasis.csv',delimiter=',',names=True,dtype=None)

In [ ]:
# Extract expression data

normal_exp_cols = list(normal_data.dtype.names[2:])
normal_exp = normal_data[normal_exp_cols].view(np.float64).reshape(normal_data.shape[0],len(normal_exp_cols))

adjacent_exp_cols = list(adjacent_data.dtype.names[2:])
adjacent_exp = adjacent_data[adjacent_exp_cols].view(np.float64).reshape(adjacent_data.shape[0],len(adjacent_exp_cols))

tumor_exp_cols = list(tumor_data.dtype.names[2:])
tumor_exp = tumor_data[tumor_exp_cols].view(np.float64).reshape(tumor_data.shape[0],len(tumor_exp_cols))

metastasis_exp_cols = list(metastasis_data.dtype.names[2:])
metastasis_exp = metastasis_data[metastasis_exp_cols].view(np.float64).reshape(metastasis_data.shape[0],len(metastasis_exp_cols))

We use a modified version of ARACNE algorithm to construct a network for each dataset.

In [ ]:
normal_rs = np.fromfile('normal-rs')
normal_rs_p = np.fromfile('normal-rs-p')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(normal_rs_p.reshape(12625*12625,1),1000,color='green',alpha=0.8)
plt.show()

In [ ]:
normal_rs = normal_rs.reshape((12625,12625))
normal_rs_p = normal_rs_p.reshape((12625,12625))
normal_rs_sig = np.where(normal_rs_p < 0.2,normal_rs,np.zeros((12625,12625)))

In [ ]:
normal_rs_adj = np.where(np.absolute(normal_rs_sig) > 0.7,normal_rs_sig,np.zeros((12625,12625)))
np.fill_diagonal(normal_rs_adj,0)
print np.transpose(np.nonzero(normal_rs_adj)).shape/2

In [ ]:
import networkx as nx


In [ ]:
graph = nx.from_numpy_matrix(normal_rs_adj)

In [ ]:
 Gc = max(nx.connected_component_subgraphs(graph), key=len)

In [ ]:
fig = plt.figure()
fig.set_size_inches(14, 9)

plt.hist(nx.degree_histogram(graph),200,color='green',alpha=0.8)
plt.show()

In [ ]:
fig = plt.figure()
fig.set_size_inches(14, 14)

nx.draw_networkx(Gc,node_size=10,alpha=0.5,)

In [ ]:
import networkx as nx
import numpy as np
from scipy.stats import spearmanr

def network(expression,pvaluefile=None,p=0.2,corr=0.7):
    if type(expression) is str:

        if type(pvaluefile) is not str:
            raise "Should provide pvaluefile."

        rs = np.fromfile(expression)
        size = np.sqrt(rs.shape[0])

        rs = rs.reshape((size,size))
        pvalue = np.fromfile(pvaluefile).reshape((size,size))
    else:
        rs,pvalue = spearmanr(expression,axis=1)

    zeros = np.zeros(rs.shape)
    rs_sig = np.where(pvalue < p,rs,zeros)
    rs_adj = np.where(np.absolute(rs_sig) > corr,rs_sig,zeros)
    np.fill_diagonal(rs_adj,0)
    G = nx.from_numpy_matrix(rs_adj)
    return G

try:
    G = network("normal-rs",pvaluefile="normal-rs-p",p=0.2,corr=0.7)
except IOError as e:
    print e
    #network(normal_exp,p=0.2,corr=0.7)
print nx.info(G)

In [ ]:
import numpy
import networkx



In [ ]:
A=numpy.matrix([[0,-0.0333],[-0.0333,0]])
print A
G=nx.from_numpy_matrix(A)
nx.info(G)

In [ ]:
import itertools
x=[1,2,3,4]
y=['a','b','c','d']
r = set()
for i in itertools.product(x,y):
    r.add(i)


In [ ]:
r